In [1]:
import pandas as pd
import numpy as np
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)

pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
import ast
import warnings
import regex as re

warnings.filterwarnings('ignore')
from decimal import *

In [2]:
# replacement={
#     '83/83':'1',    
# }

# # Replacement
# multi[curation_col]= multi[curation_col].apply(
#     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

# matches=not_tube
# matches[curation_col]=matches['height'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'7\/165','.44',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))


# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return f'{lower}" - {upper}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return f'{original_val}"{metric}'
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches[curation_col]=matches['Min'].apply(range_app)
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches


# # Find a list of values to iterate
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# # list_values.append('coarse')

# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_values)
# # Extract Key Words
# df['matches']=df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))  

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]  

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))    

def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

# str(round(num, 2))
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers

def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

In [4]:
customer_id = '137'
customer_name='%kimballwest%'
buckets = "Fixings & Fasteners"
strategy_version_input=705
attribute_id_input=4892
formatted_attribute = 'Maximum Operating Temperature'

In [5]:
# parameters
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [6]:
buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
WHERE sba.attribute_id = {attribute_id_input} 
AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()] 

In [7]:
df = query_from_file(file_name='./query/uncurated.sql', params=params)
print(len(df)) 
df = df[df['bucket_id'].isin(buckets)]
height=df
len(df)

17008


17008

In [8]:
df['buckets'].explode().value_counts()[0:50] 

Fixings & Fasteners                                                16665
Electrical System                                                    342
Cables, Cords & Cord Management Replacement Parts & Accessories        1
Name: buckets, dtype: int64

In [9]:
df[df['external_id'].astype(str)=='91228']

product_id external_id            product_name  \
2880  1173673346       91228  1-1/2 x 1" PVC Bushing   

                                                                                                                                                                                                                                                                                                                                                                                                      long_desc  \
2880  Available in PVC (PolyVinyl Chloride) and CPVC (Chlorinated PolyVinyl Chloride) material. Maximum operating temperature for PVC systems should not exceed 140°F (110°F threaded systems). Maximum operating temperature for CPVC systems should not exceed 200°F (150°F threaded systems). PVC & CPVC use different sizing systems. A 1/2" PVC cap is larger than a 1/2" CPVC cap & will not interchange.   

      customer_id attributes              buckets  bucket_id values  
2880          137       None  Fixings & Fasteners       7265   None

In [10]:
df_fasteners=df[df['buckets'].astype(str)=='Fixings & Fasteners']
print(len(df_fasteners))
# df_fasteners

16665


In [11]:
regex_pattern=r'(?i)(systems should not exceed .{0,7}°F)|(rated performance up to .{0,7}°F)|(Heat stable to .{0,7}°F)|(Maximum temperature.{20})|(systems should not exceed.{7})|(Maximum operating temperature: PVC =.{7})|(CPVC \= .{7})|(temperatures as high as.{7})|(Full rated performance up to.{7})|°F to.{7|()'
df_fasteners['matches']=df_fasteners['long_desc'].apply(lambda x: re_extract(regex_pattern, x)) 
print(len(df_fasteners[df_fasteners['matches'].astype(str)!='[]'])) 
minimum=df_fasteners[df_fasteners['matches'].astype(str)!='[]']
minimum

150


product_id external_id  \
316    1173659452       85531   
317    1173659447       85501   
318    1173659438       85517   
319    1173659432       85541   
469    1173659455       85507   
470    1173659453       85538   
471    1173659448       85500   
472    1173659445       85519   
473    1173659440       85545   
474    1173659439       85503   
475    1173659434       85505   
476    1173659428       85518   
2868   1173673449       92191   
2869   1173673447       92588   
2870   1173673436       92592   
2871   1173673431       92192   
2872   1173673428       92585   
2873   1173673419       92596   
2875   1173673414       92583   
2876   1173673409       92584   
2877   1173673407       92587   
2879   1173673377       92594   
2880   1173673346       91228   
2883   1173673343       91233   
2884   1173673339       91226   
2885   1173673338       91188   
2886   1173673335       91185   
2888   1173673332       91231   
2889   1173673329       91229   
2890   1173673328       91184   
2891   1173673326       91232   
2893   1173673317       91183   
2932   1173673510       92201   
2934   1173673503       92193   
2935   1173673497       92195   
2936   1173673490       92186   
2938   1173673481       92189   
2939   1173673476       92591   
2940   1173673470       92590   
2941   1173673463       92580   
2942   1173673458       92597   
2943   1173673456       92188   
2944   1173673454       92184   
2974   1173672850       92788   
2975   1173672838       92796   
2976   1173672833       92387   
2977   1173672801       92385   
2978   1173672770       92397   
2979   1173672767       92391   
2980   1173672745       91373   
3064   1173666410       50802   
3065   1173666403       50807   
4138   1173660668      375032   
4827   1173659461       85510   
5400   1173659451       85506   
5939   1173659433       85522   
5953   1173660310      375207   
6010   1173660673      375026   
6016   1173660674      375028   
6018   1173660666      375030   
6025   1173660309      375205   
6036   1173660311      375204   
6226   1173659462       85511   
6252   1173660672      375024   
6254   1173659441       85516   
6408   1173659458       85509   
6418   1173660663      375034   
6433   1173660657      375023   
6478   1173673501       92187   
6479   1173673435       92194   
6480   1173672854       92795   
6481   1173672853       92791   
6486   1173672849       92787   
6487   1173672842       92785   
6488   1173672829       92396   
6489   1173672803       92783   
6741   1173660667      375031   
6742   1173660665      375029   
6743   1173660664      375039   
6772   1173660312      375200   
6773   1173660301      375212   
6882   1173659459       85508   
6883   1173659456       85544   
6884   1173659449       85504   
6885   1173659444       85532   
6886   1173659437       85502   
6887   1173659430       85523   
10108  1173666412       50808   
10109  1173666411       50804   
10637  1173647757      408551   
11001  1173673483       92196   
11002  1173673347       91186   
11122  1173660662      375036   
11123  1173660661      375033   
11124  1173660659      375021   
11125  1173660654      375041   
11141  1173660308      375214   
11142  1173660306      375202   
11143  1173660303      375217   
11144  1173660302      375210   
11145  1173660298      375213   
11146  1173660297      375209   
11308  1173659435       85542   
11453  1173673499       92581   
11454  1173673496       92190   
11455  1173673459       92197   
11456  1173673452       92183   
11457  1173672851       92790   
11458  1173672831       92784   
11459  1173672781       92389   
11564  1173660670      375022   
11895  1173660669      375025   
11913  1173660313      375201   
12200  1173672839       92789   
12201  1173672826       92384   
12202  1173673327       91187   
12203  1173673324       91230   
12210  1173666409       50810   
12312  1173660300      375208   
12644  1173659463       85535   
12645  1173659

In [12]:
regex_pattern=r'(?i)(-\d+°F to)|()'
# minimum['matchez']=minimum['matches'].apply(lambda x: re_extract(regex_pattern, x)) 
minimum['matchez']=minimum['matches'].apply(lambda x: re.sub(r'[a-z]|[A-Z]|°|\.|=|\\| |\:','',str(x))).apply(lambda x: re.sub(r"0140",'140',str(x))).apply(lambda x: re.sub(r"'",'"',str(x)))
minimum['matchez'].explode().value_counts()
# minimum[curation_col]=minimum['matches']
# matches=minimum[['external_id',curation_col]]
# matches=matches[matches[curation_col].astype(str)!='']
minimum[curation_col]=minimum['matchez']
matches=minimum[['external_id',curation_col]]
matches

external_id Q:maximum_operating_temperature
316         85531                         ["300"]
317         85501                         ["300"]
318         85517                         ["300"]
319         85541                         ["300"]
469         85507                         ["300"]
470         85538                         ["300"]
471         85500                         ["300"]
472         85519                         ["300"]
473         85545                         ["300"]
474         85503                         ["300"]
475         85505                         ["300"]
476         85518                         ["300"]
2868        92191                   ["140","210"]
2869        92588                   ["140","210"]
2870        92592                   ["140","210"]
2871        92192                   ["140","210"]
2872        92585                   ["140","210"]
2873        92596                   ["140","210"]
2875        92583                   ["140","210"]
2876        92584                   ["140","210"]
2877        92587                   ["140","210"]
2879        92594                   ["140","210"]
2880        91228                   ["140","200"]
2883        91233                   ["140","200"]
2884        91226                   ["140","200"]
2885        91188                   ["140","200"]
2886        91185                   ["140","200"]
2888        91231                   ["140","200"]
2889        91229                   ["140","200"]
2890        91184                   ["140","200"]
2891        91232                   ["140","200"]
2893        91183                   ["140","200"]
2932        92201                   ["140","210"]
2934        92193                   ["140","210"]
2935        92195                   ["140","210"]
2936        92186                   ["140","210"]
2938        92189                   ["140","210"]
2939        92591                   ["140","210"]
2940        92590                   ["140","210"]
2941        92580                   ["140","210"]
2942        92597                   ["140","210"]
2943        92188                   ["140","210"]
2944        92184                   ["140","210"]
2974        92788                   ["140","210"]
2975        92796                   ["140","210"]
2976        92387                   ["140","210"]
2977        92385                   ["140","210"]
2978        92397                   ["140","210"]
2979        92391                   ["140","210"]
2980        91373                   ["140","200"]
3064        50802                         ["470"]
3065        50807                         ["470"]
4138       375032                         ["230"]
4827        85510                         ["300"]
5400        85506                         ["300"]
5939        85522                         ["300"]
5953       375207                         ["185"]
6010       375026                         ["230"]
6016       375028                         ["230"]
6018       375030                         ["230"]
6025       375205                         ["185"]
6036       375204                         ["185"]
6226        85511                         ["300"]
6252       375024                         ["230"]
6254        85516                         ["300"]
6408        85509                         ["300"]
6418       375034                         ["230"]
6433       375023                         ["230"]
6478        92187                   ["140","210"]
6479        92194                   ["140","210"]
6480        92795                   ["140","210"]
6481        92791                   ["140","210"]
6486        92787                   ["140","210"]
6487        92785                   ["140","210"]
6488        92396                   ["140","210"]
6489        92783                   ["140","210"]
6741       375031                         ["230"]
6742       375029                         ["230"]
6743       375039                         ["230"]
6772  

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',wipe)

# send to the folder for upload

In [18]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
    matches.to_csv(f'{drive_path}/Fasteners- {attribute}-- value- upload_Shawn.csv',index=False)

In [19]:
looks_good('Kimball Midwest', attribute, df, matches)

In [25]:
def looks_good_na(customer, attribute, null, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = null[null['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/Fasteners - {attribute} - na upload.csv',index=False)
#     matches.to_csv(f'{drive_path}/Fasteners - {attribute}-- total_na_values upload_Shawn.csv',index=False)

In [26]:
looks_good_na('Kimball Midwest', attribute, df, final)